In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
%aimport sys
sys.path.insert(0,'..')

In [4]:
%env TEXTMORPH_DATA=/Users/guillaume/Desktop/backup/neural-editor-grll-data

env: TEXTMORPH_DATA=/Users/guillaume/Desktop/backup/neural-editor-grll-data


In [5]:
from os.path import dirname, realpath, join

from textmorph.text_generation.training_run import EditTrainingRuns
from textmorph import data
from textmorph.text_generation.training_run import GenData
from textmorph.text_generation.decoder import TrainDecoderInput
from gtd.ml.torch.utils import similar_size_batches
from gtd.chrono import verboserate

In [6]:
data_dir = join(data.workspace.root, "de-amazon-review")

In [13]:
source_data = GenData(data_dir)

[localhost] local: wc -l /Users/guillaume/Desktop/backup/neural-editor-grll-data/de-amazon-review/valid.tsv


In [14]:
source_data.data # match well

[SOURCE: meine erwartungen an das das empfehlungsmarketing wurden nicht enttäuscht .,
 SOURCE: ich kann mich meinen vorredner nur anschließen .,
 SOURCE: was bitte hat das mit der handlung zu tun ?,
 SOURCE: schlechtester film , den ich je gesehen habe,
 SOURCE: ich kann dieses buch nur weiterempfehlen !,
 SOURCE: und da kann ich nur sagen :,
 SOURCE: ist dieses buch von <per> .,
 SOURCE: <ordinal> sterne für die dvd . <org>,
 SOURCE: ich kann dieses buch nur empfehlen . <org>,
 SOURCE: ich wurde aber leider maßlos enttäuscht .,
 SOURCE: er ist einfach nur belanglos .,
 SOURCE: dieses buch war mein erstes buch von <org> .,
 SOURCE: es ist flüssig und spannend geschrieben .,
 SOURCE: ich bin wirklich sehr enttäuscht von diesem buch .,
 SOURCE: dieses album ist ein muss für jeden techno-freak .,
 SOURCE: dieser film ist einfach nur schlecht .,
 SOURCE: <per> vom anfang bis zum ende .,
 SOURCE: ich hab nach <ordinal> seiten aufgehört dieses buch zu lesen .,
 SOURCE: die autorin <per> hat 

In [ ]:
experiments = EditTrainingRuns(check_commit=False)
exp_id = [6]
exp = experiments[int(exp_id[0])]

In [ ]:
batches = similar_size_batches(source_data.data, 32, size=lambda x: len(x.source_words))

In [ ]:
for batch in verboserate(batches, desc='Streaming Source Sentences'):
    # Source Encode
    source_words, insert_words, insert_exact_words, delete_words, delete_exact_words, target_words, edit_embed = exp.editor._batch_editor_examples(batch)
    insert_exact_words = [[u'war']]*32
    delete_exact_words = [[u'bin']]*32
    encoder_input = exp.editor.encoder.preprocess(source_words, insert_words, insert_exact_words, delete_words, delete_exact_words, edit_embed)
    encoder_output = exp.editor.encoder(encoder_input)

    break

In [ ]:
source_words[0]

In [ ]:
delete_exact_words[0]

In [ ]:
insert_exact_words[0]

In [ ]:
encoder_input.source_words.values[0]

In [ ]:
encoder_input.insert_exact_words.values[0]

In [ ]:
encoder_input.delete_exact_words.values[0]

In [ ]:
encoder_output.source_embeds.values[0].shape

In [ ]:
train_decoder_input = TrainDecoderInput(32, exp.editor.train_decoder.word_vocab)

In [ ]:
import torch
from gtd.ml.torch.seq_batch import SequenceBatch
from gtd.ml.torch.utils import GPUVariable

word_vocab = exp.editor.train_decoder.word_vocab

# decoder forward:
batch_size, _ = encoder_output.source_embeds.mask.size()
rnn_state = exp.editor.train_decoder.decoder_cell.initialize(batch_size)

input_word_embeds = exp.editor.train_decoder.token_embedder.embed_seq_batch(train_decoder_input.input_words)

input_embed_list = input_word_embeds.split()

loss_list = []
rnn_states = []

generated_sentences = []

values = input_embed_list[0].values
mask = input_embed_list[0].mask

while mask.nonzero().size()[0] != 0:
    rnn_input = exp.editor.train_decoder.rnn_context_combiner(encoder_output, values)

    decoder_cell_output = exp.editor.train_decoder.decoder_cell(rnn_state, rnn_input, mask)
    rnn_state = decoder_cell_output.rnn_state
    rnn_states.append(rnn_state)
        
    greedy_values = torch.max(decoder_cell_output.vocab_probs, dim=1)[1]
    generated_sentences.append(word_vocab.indices2words(greedy_values))
    print(generated_sentences[0])
    
    values = greedy_values.unsqueeze(1)
    
    mask = greedy_values.apply_(lambda x: 0. if x == word_vocab.STOP else 1.).unsqueeze(1)  
    sq = SequenceBatch(GPUVariable(values), GPUVariable(mask))
    sq_embeded = exp.editor.train_decoder.token_embedder.embed_seq_batch(sq)
    
    values = sq_embeded.split()[0].values
    mask = sq_embeded.split()[0].mask.type(torch.FloatTensor)
    
    break

In [ ]:
torch.topk(decoder_cell_output.vocab_probs[0], 10)

In [ ]:
word_vocab.indices2words(torch.topk(decoder_cell_output.vocab_probs[0], 20)[1].tolist())

In [ ]:
torch.max(decoder_cell_output.vocab_probs[0], dim=0)

In [ ]:
torch.max(decoder_cell_output.vocab_probs, dim=1)[1]

In [ ]:
greedy_values

In [ ]:
word_vocab.index2word(1)

In [ ]:
mask = greedy_value.apply_(lambda x: 0.0 if x == word_vocab.STOP else 1.0).unsqueeze(1)
mask.shape

In [ ]:
generated_sentences

In [ ]:
input_word_embeds = exp.editor.train_decoder.token_embedder.embed_seq_batch(test)

In [ ]:
input_word_embeds

In [ ]:
from gtd.ml.torch.seq_batch import SequenceBatch
from gtd.ml.torch.utils import GPUVariable

test = SequenceBatch(GPUVariable(values), GPUVariable(mask))

In [ ]:
train_decoder_input.input_words